In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
#website help from 
#https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

In [3]:
df = pd.read_csv('df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [4]:
def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 8),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 10, 20)
    }

    return XGBRegressor(**params)

def instantiate_gbr(trial : Trial) -> GradientBoostingRegressor:
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'max_depth': trial.suggest_int ('max_depth', 2, 6),
    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    'min_samples_split': trial.suggest_int ('min_sample_split', 6, 10),
    'min_samples_leaf': trial.suggest_int ('min_sample_leaf', 1, 10)
    }
    
    return GradientBoostingRegressor(**params)



def instantiate_ada(trial : Trial) -> AdaBoostRegressor:
    params = {
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'loss': trial.suggest_categorical('loss',['linear', 'square', 'exponential']),
    'n_estimators': trial.suggest_int('n_estimators', 1, 1000)
    }
    
    return AdaBoostRegressor(**params)



In [5]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_ordinal_encoder(trial: Trial)-> OrdinalEncoder:
    params = {
        'handle_unknown': "use_encoded_value", 
        'unknown_value': -1
    }
    
    return OrdinalEncoder(**params)

def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': trial.suggest_categorical('drop', [None, 'first'])
    }
    
    return OneHotEncoder(**params)
    
Encoder = (
    OrdinalEncoder |
    OneHotEncoder 
    )

def instantiate_encoder (trial : Trial) -> Encoder:
    encoding_method = trial.suggest_categorical(
        'encoding_method', ['ordinal', 'onehot'])
    if encoding_method =='ordinal':
        encoder = instantiate_ordinal_encoder(trial)
    elif encoding_method =='onehot':
        encoder = instantiate_onehot_encoder(trial)
    
    return encoder

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor



In [21]:
def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True)
    }

    return Lasso(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_lasso(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_lasso(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [22]:
from optuna import create_study

study_lasso = create_study(study_name='optimization', direction='maximize')

study_lasso.optimize(lambda trial: objective_lasso(trial, X, y), n_trials=500)

[I 2024-05-29 14:51:13,036] A new study created in memory with name: optimization
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2024-05-29 14:51:24,571] Trial 9 finished with value: 0.9488496888258731 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': 'first', 'alpha': 5.6491904998249895e-05}. Best is trial 4 with value: 0.9505733771926941.
[I 2024-05-29 14:51:25,307] Trial 10 finished with value: 0.9415487212016689 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0008314848717242383}. Best is trial 4 with value: 0.9505733771926941.
[I 2024-05-29 14:51:27,749] Trial 11 finished with value: 0.9499038113387861 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 7.324482262516736e-05}. Best is trial 4 with value: 0.9505733771926941.
C:\Users\Natal\anaconda3\Lib\sit

[I 2024-05-29 14:51:55,584] Trial 29 finished with value: 0.9506292681741411 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0002060937982355667}. Best is trial 29 with value: 0.9506292681741411.
[I 2024-05-29 14:51:57,069] Trial 30 finished with value: 0.9482729842560413 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00020325969886929256}. Best is trial 29 with value: 0.9506292681741411.
[I 2024-05-29 14:51:58,973] Trial 31 finished with value: 0.9502309819358927 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 9.69126308266606e-05}. Best is trial 29 with value: 0.9506292681741411.
[I 2024-05-29 14:52:00,214] Trial 32 finished with value: 0.9506434379446984 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00022205724535290372}. Best is trial 32 with value: 0.9506434379446984.
[I 2024-05-

[I 2024-05-29 14:52:16,311] Trial 54 finished with value: 0.9507421606088157 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003872624804640623}. Best is trial 42 with value: 0.9508124564356155.
[I 2024-05-29 14:52:17,243] Trial 55 finished with value: 0.950812505262452 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00030728661198099863}. Best is trial 55 with value: 0.950812505262452.
[I 2024-05-29 14:52:17,915] Trial 56 finished with value: 0.9503742664155641 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0005618370120886315}. Best is trial 55 with value: 0.950812505262452.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\sit

[I 2024-05-29 14:52:36,092] Trial 74 finished with value: 0.9508075407695149 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00033323311960732106}. Best is trial 55 with value: 0.950812505262452.
[I 2024-05-29 14:52:36,902] Trial 75 finished with value: 0.9508070926682036 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.000333978668248268}. Best is trial 55 with value: 0.950812505262452.
[I 2024-05-29 14:52:37,673] Trial 76 finished with value: 0.950590807411794 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00047184222773295385}. Best is trial 55 with value: 0.950812505262452.
[I 2024-05-29 14:52:38,552] Trial 77 finished with value: 0.9507902523371292 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00035379627368347787}. Best is trial 55 with value: 0.950812505262452.
[I 2024-05-29 1

...

[I 2024-05-29 14:57:15,597] Trial 390 finished with value: 0.9480519486113904 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00026500212946237774}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:57:16,465] Trial 391 finished with value: 0.9508066520310324 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003346619256468453}. Best is trial 132 with value: 0.9508125912427354.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packag

[I 2024-05-29 14:57:37,147] Trial 414 finished with value: 0.9505625074535293 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00021663869256331072}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:57:38,026] Trial 415 finished with value: 0.9507493730081743 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003825979314932795}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:57:38,922] Trial 416 finished with value: 0.9508076538197476 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.000333002867791055}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:57:39,822] Trial 417 finished with value: 0.9508093946663341 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00029424425171423305}. Best is trial 132 with value: 0.9508125912427354.
[I 

[I 2024-05-29 14:58:01,651] Trial 436 finished with value: 0.9507993669771452 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0002832868627250655}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:58:02,452] Trial 437 finished with value: 0.9507384610242084 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003897437122896452}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:58:03,857] Trial 438 finished with value: 0.9482182527683751 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00023061492352636473}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:58:04,767] Trial 439 finished with value: 0.9508104544280058 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003263611385344029}. Best is trial 132 with value: 0.9508125912427354.
[I 

[I 2024-05-29 14:58:26,867] Trial 465 finished with value: 0.9417424322770802 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.0002972402260064716}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:58:27,670] Trial 466 finished with value: 0.9507712919173523 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003670688223050775}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:58:28,460] Trial 467 finished with value: 0.9508058597979732 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00033582259771325264}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:58:29,322] Trial 468 finished with value: 0.9507427628339128 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.000258397795074041}. Best is trial 132 with value: 0.9508125912427354.
[I 2024-05-29 14:

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

In [23]:
print(study_lasso.best_value)
print(study_lasso.best_params)

0.9508125912427354
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00030615490713200075}


In [17]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 1, 100, log=True)
    }

    return Ridge(**params)


def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ridge(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ridge(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [20]:
from optuna import create_study

study_ridge = create_study(study_name='optimization', direction='maximize')

study_ridge.optimize(lambda trial: objective_ridge(trial, X, y), n_trials=500)

[I 2024-05-29 14:46:39,540] A new study created in memory with name: optimization
[I 2024-05-29 14:46:39,890] Trial 0 finished with value: 0.9489513810750617 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 4.224215714175673}. Best is trial 0 with value: 0.9489513810750617.
[I 2024-05-29 14:46:40,182] Trial 1 finished with value: 0.9447229219725486 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 5.62343318358076}. Best is trial 0 with value: 0.9489513810750617.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2024-05-29 14:46:44,720] Trial 14 finished with value: 0.9493767434924593 and para

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2024-05-29 14:46:48,215] Trial 20 finished with value: 0.9495111507934821 and para

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

...

[I 2024-05-29 14:49:59,759] Trial 366 finished with value: 0.9500462113969107 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 21.15105104807232}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:00,221] Trial 367 finished with value: 0.9283206508025575 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 25.1504862293041}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:00,787] Trial 368 finished with value: 0.9500110328660295 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 17.52028749586782}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:01,329] Trial 369 finished with value: 0.9500481858201946 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 22.755778854758248}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:

[I 2024-05-29 14:50:15,249] Trial 395 finished with value: 0.9475413800912751 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 99.24229776417987}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:15,790] Trial 396 finished with value: 0.9500256193251395 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 26.766538098045515}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:16,159] Trial 397 finished with value: 0.9215942968962885 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'alpha': 18.465677208472446}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:16,689] Trial 398 finished with value: 0.9500416876304545 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 20.23582942346699}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:17,274] Tri

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

[I 2024-05-29 14:50:44,388] Trial 449 finished with value: 0.9499776726361617 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 15.888134595692799}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:44,948] Trial 450 finished with value: 0.9480757075307071 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 1.7057935050670603}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:45,555] Trial 451 finished with value: 0.9500413413153007 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 20.182452332392728}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:50:46,087] Trial 452 finished with value: 0.9500086293071217 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 17.378643925913124}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14

[I 2024-05-29 14:51:01,044] Trial 478 finished with value: 0.9499295609559629 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 14.236213475051581}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:51:01,700] Trial 479 finished with value: 0.9500382644345023 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 19.759927928719005}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:51:02,289] Trial 480 finished with value: 0.950013226738944 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 27.908381542438477}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14:51:02,759] Trial 481 finished with value: 0.9496845724984592 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'alpha': 23.46890916959578}. Best is trial 207 with value: 0.9500483603074065.
[I 2024-05-29 14

In [24]:
print(study_ridge.best_value)
print(study_ridge.best_params)

0.9500483603074065
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 22.384020959927625}


# EN

In [64]:
def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', .1, .7, log = True)
    }

    return ElasticNet(**params)



def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_en(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_en(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [65]:
from optuna import create_study

study_en = create_study(study_name='optimization', direction='maximize')

study_en.optimize(lambda trial: objective_en(trial, X, y), n_trials=200)

[I 2024-06-04 12:11:09,318] A new study created in memory with name: optimization
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.404e+00, tolerance: 2.409e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.523e+00, tolerance: 2.445e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_e

[I 2024-06-04 12:11:32,673] Trial 14 finished with value: 0.9417705577027908 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'ordinal', 'alpha': 0.00012726945204769044, 'l1_ratio': 0.6973862163128914}. Best is trial 13 with value: 0.9503906980010737.
[I 2024-06-04 12:11:33,538] Trial 15 finished with value: 0.9504647308835044 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00033039138120244133, 'l1_ratio': 0.5839963510057812}. Best is trial 15 with value: 0.9504647308835044.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.265e-02, tolerance: 2.383e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: Convergen

[I 2024-06-04 12:11:55,895] Trial 37 finished with value: 0.9488350820536899 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00024328192657776227, 'l1_ratio': 0.13196826812856255}. Best is trial 31 with value: 0.9505499049761467.
[I 2024-06-04 12:11:56,401] Trial 38 finished with value: 0.9417457762066904 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.0007354764476121989, 'l1_ratio': 0.3159175589563822}. Best is trial 31 with value: 0.9505499049761467.
[I 2024-06-04 12:11:57,104] Trial 39 finished with value: 0.9481549613128278 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0004022539129003468, 'l1_ratio': 0.5374274671812856}. Best is trial 31 with value: 0.9505499049761467.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown ca

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

...

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

[I 2024-06-04 12:13:02,986] Trial 112 finished with value: 0.9508142864190117 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0008538883065837565, 'l1_ratio': 0.33578661444506}. Best is trial 110 with value: 0.950820843082717.
[I 2024-06-04 12:13:03,759] Trial 113 finished with value: 0.9508191540538323 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0008739171477669513, 'l1_ratio': 0.3401917031329937}. Best is trial 110 with value: 0.950820843082717.
[I 2024-06-04 12:13:04,545] Trial 114 finished with value: 0.9508140305778845 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0008920977328144325, 'l1_ratio': 0.32063652487224986}. Best is trial 110 with value: 0.950820843082717.
[I 2024-06-04 12:13:05,307] Trial 115 finished with value: 0.9508206424132493 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': N

[I 2024-06-04 12:13:25,967] Trial 142 finished with value: 0.9508105015577785 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0007872105072179762, 'l1_ratio': 0.3580349217835481}. Best is trial 110 with value: 0.950820843082717.
[I 2024-06-04 12:13:26,699] Trial 143 finished with value: 0.9508202201885936 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0009044251245493753, 'l1_ratio': 0.3330619199452988}. Best is trial 110 with value: 0.950820843082717.
[I 2024-06-04 12:13:27,447] Trial 144 finished with value: 0.9507954368725164 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0009265280516420815, 'l1_ratio': 0.37660970968558816}. Best is trial 110 with value: 0.950820843082717.
[I 2024-06-04 12:13:28,239] Trial 145 finished with value: 0.9506670212463165 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop':

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.595e+00, tolerance: 2.409e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.520e+00, tolerance: 2.445e-02
  model = cd_fast.enet_coordinate_descent(
[I 2024-06-04 12:13:47,685] Trial 168 finished with value: 0.9409312411500317 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 1.920788613331729e-05, 'l1_ratio': 0.2803920883031563}. Best is trial 110 with value: 0.950820843082717.
[I 2024-06-04 12:13:48,373] T

[I 2024-06-04 12:14:06,796] Trial 189 finished with value: 0.9508211865943965 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0007890891316732335, 'l1_ratio': 0.386603322661332}. Best is trial 189 with value: 0.9508211865943965.
[I 2024-06-04 12:14:07,632] Trial 190 finished with value: 0.9508115073280979 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0007539799779360345, 'l1_ratio': 0.3758948220644501}. Best is trial 189 with value: 0.9508211865943965.
[I 2024-06-04 12:14:08,462] Trial 191 finished with value: 0.9508125388598604 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0008330956165337611, 'l1_ratio': 0.3987889471811326}. Best is trial 189 with value: 0.9508211865943965.
[I 2024-06-04 12:14:09,309] Trial 192 finished with value: 0.9508210288482131 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop'

In [66]:
# after adding in the L1 value
print(study_en.best_value)
print(study_en.best_params)

0.9508214083737423
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006921322276640134, 'l1_ratio': 0.4415555448483296}


In [29]:
print(study_en.best_value)
print(study_en.best_params)

0.950820633330468
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006178621404820522}


In [67]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

en = ElasticNet(alpha = 0.0006178621404820522, l1_ratio= .5)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")

numerical_preprocessor = RobustScaler()

preprocessor = ColumnTransformer(
    [
        ("robust-scaler", numerical_preprocessor, numerical_columns),
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
    ]
)

en_pipe = Pipeline(steps=[("Preprocess", preprocessor), ("EN", en)])

en_regr = compose.TransformedTargetRegressor(regressor= en_pipe,
                                                func=np.log, inverse_func=np.exp)

In [68]:
en_scores = cross_val_score(en_regr, X, y, cv = kf, error_score= 'raise')
en_scores.mean()
#0.950820633330468

0.950820633330468

# RF

In [48]:
def instantiate_rf(trial : Trial) -> RandomForestRegressor:
    params = {
    'bootstrap':trial.suggest_categorical('bootstrap', [True]),
    'n_estimators': trial.suggest_int('n_estimators', 250, 500),
    'max_depth': trial.suggest_int('max_depth', 50, 75),
    'min_samples_split': trial.suggest_int('min_sample_split', 4, 6),
    'min_samples_leaf': trial.suggest_int('min_sample_leaf', 1, 5)
    }
    
    return RandomForestRegressor(**params)
  
def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_rf(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_rf(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [49]:
from optuna import create_study

study_rf = create_study(study_name='optimization', direction='maximize')

study_rf.optimize(lambda trial: objective_en(trial, X, y), n_trials=50)

[I 2024-05-31 08:58:37,051] A new study created in memory with name: optimization
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

[I 2024-05-31 10:31:29,412] Trial 32 finished with value: 0.9094899639883157 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 327, 'max_depth': 63, 'min_sample_split': 6, 'min_sample_leaf': 1}. Best is trial 25 with value: 0.9104726627980002.
[I 2024-05-31 10:34:00,216] Trial 33 finished with value: 0.9095126002045921 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 277, 'max_depth': 67, 'min_sample_split': 5, 'min_sample_leaf': 1}. Best is trial 25 with value: 0.9104726627980002.
[I 2024-05-31 10:36:38,891] Trial 34 finished with value: 0.9102377098381838 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 305, 'max_depth': 71, 'min_sample_split': 4, 'min_sample_leaf': 2}. Best is trial 25 with value: 0.9104726627980002.
[I 2024-05-31 10:38:43,955] Trial 35 finished with value: 0.9064976064216932 and paramet

In [51]:
print(study_rf.best_value)
print(study_rf.best_params)

0.9106961840498735
{'scaling_method': 'robust', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 338, 'max_depth': 73, 'min_sample_split': 4, 'min_sample_leaf': 2}


# XGB